# Chapter 4: Create a new environment 

It's time to make your own environment!

Here, you will learn how to create your own environment for the **Gym_Gazebo** module. For other systems, you just do the same steps, but in the general Gym module or your own custom version, similar to Gym_Gazebo, instead.

## Step 1: Know your environment

**Learn what the possible inputs are for the control of your system. The actions it's able to perform.<br>
Learn what kind of environmental data is available for learning.<br>
What do you want to teach to your robot?<br>**

In this case, you will be using a Parrot AR.Drone simulation. The drone has to learn how to fly low so that enemy drones can't detect it. But, it has to also know how to go over barriers without exceeding that maximum elevation or hitting them.

The sensors that will be used at the begining will be a **compass** and an **altitude sensor**.<br>
The system will be controlled through five actions: **FORWARD, TURN LEFT, TURN RIGHT, GO UP, and GO DOWN.**<br>

But first, let's test how the drone performs a little bit:

First, you need it to take off.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic pub /drone/takeoff std_msgs/Empty "{}"

Hit **CTRL+C** to stop it and be able to type more commands. In this case, the command is to move the drone with the keyboard.

Execute the following command in order to start a ROS program that will allow you to control the drone through the keybord.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun teleop_twist_keyboard teleop_twist_keyboard.py

The fpllowing are the basic keys used to control the drone:

<table style="width:100%">
  <tr>
    <th>
    <figure>
        <img src="img/key_1.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Take-off</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_2.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Land</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_i.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Move forward</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_comma.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Move backward</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_j.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Turn left</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_l.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Turn right</p>
    </th> 
  </tr>
  <tr>
    <th>
    <figure>
        <img src="img/key_k.png"width="40"></img>
        
    </figure>
    </th>
    <th>
    <p style="text-align: center;">Stop</p>
    </th> 
  </tr>
</table>

To land the drone, just publish this into the **/drone/land** topic:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic pub /drone/land std_msgs/Empty "{}"

## Step 2: What you have to edit and create

You will now edit the Gym_Gazebo Python module, where all the enviroments are set up. Outside Gazebo, it would be the gym module. So, go to the following path: **/usr/local/lib/python2.7/dist-packages/gym\_gazebo**.

Inside, you will have to edit these two files:

**\_\_init\_\_.py**<br>
**scoreboard/\_\_init\_\_.py**

You will have to also create a new environment script inside the **envs** folder.<br>
See how it's already done a first version for you.

<p style="background:#EE9023;color:white;">**Exercise 4.1**</p>

Edit the **\_\_init\_\_.py** and **scoreboard/\_\_init\_\_.py** scripts to add a new **QuadCopter-v1** environment.

<p style="background:#CD2122;color:white;">**WARNING**</p><br>
Keep in mind that a very common error afterwards is that your learning script will sometimes give an error, stating that you can't reset, if the episode is not done yet. This is because your timelimit and the number of iterations don't correspond. So be careful with that.<br>
<p style="background:#CD2122;color:white;">**WARNING**</p>

Once it's done, you have to create a new environment file inside the **envs** folder. You have an example file in the following path: **/usr/local/lib/python2.7/dist-packages/gym\_gazebo/envs/gazebo_quadcopter.py**

<p style="background:green;color:white;">**gazebo_quadcopter.py**</p>

In [ ]:
import gym
import rospy
import roslaunch
import time
import numpy as np
import tf
import time
from gym import utils, spaces
from gym_gazebo.envs import gazebo_env
from geometry_msgs.msg import Twist, Vector3Stamped, Pose
from std_srvs.srv import Empty
from hector_uav_msgs.msg import Altimeter
from sensor_msgs.msg import Imu
from std_msgs.msg import Empty as EmptyTopicMsg
from gym.utils import seeding

class QuadCopterEnv(gazebo_env.GazeboEnv):

    def __init__(self):
        # Launch the simulation with the given launchfile name
        gazebo_env.GazeboEnv.__init__(self, "GazeboCircuitTurtlebotLidar_v0.launch")
        self.vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=5)
        self.takeoff_pub = rospy.Publisher('/drone/takeoff', EmptyTopicMsg, queue_size=None)
         
        self.unpause = rospy.ServiceProxy('/gazebo/unpause_physics', Empty)
        self.pause = rospy.ServiceProxy('/gazebo/pause_physics', Empty)
        self.reset_proxy = rospy.ServiceProxy('/gazebo/reset_simulation', Empty)
        
        self.action_space = spaces.Discrete(5) #F,L,R
        self.reward_range = (-np.inf, np.inf)

        self._seed()


    def calculate_dist_between_two_Points(self,p_init,p_end):
        a = np.array((p_init.x ,p_init.y, p_init.z))
        b = np.array((p_end.x ,p_end.y, p_end.z))
        
        dist = np.linalg.norm(a-b)
        
        return dist

    def get_init_pose(self):
        data_pose = None
        while data_pose is None:
            try:                
                data_pose = rospy.wait_for_message('/drone/gt_pose', Pose, timeout=1)
            except:
                rospy.loginfo("Current drone pose not ready yet, retrying for setting up init pose")
        return data_pose


    def init_desired_pose(self):
        self.desired_pose = Pose()
        self.desired_pose.position.z = 0.7
        self.desired_pose.position.x = 14.0
        self.desired_pose.position.y = 1.0
        current_init_pose = self.get_init_pose()
        
        self.best_dist = self.calculate_dist_between_two_Points(current_init_pose.position, self.desired_pose.position)
    

    def check_topic_publishers_connection(self):
        
        rate = rospy.Rate(100) # 1hz
        while(self.takeoff_pub.get_num_connections() == 0):
            rospy.loginfo("No suscribers to take off yet, so we'll wait and try again")
            rate.sleep();
        rospy.loginfo("Takeoff Publisher Connected")

        while(self.vel_pub.get_num_connections() == 0):
            rospy.loginfo("No subscribers to Cmd_vel yet so we'll wait and try again")
            rate.sleep();
        rospy.loginfo("Cmd_vel Publisher Connected")
        

    def reset_cmd_vel_commands(self):
        # We send an empty null Twist
        vel_cmd = Twist()
        vel_cmd.linear.z = 0.0
        vel_cmd.angular.z = 0.0
        self.vel_pub.publish(vel_cmd)


    def takeoff_sequence(self, seconds_taking_off=1):
        # Before taking off, be sure that cmd_vel value there is is null to avoid drifts
        self.reset_cmd_vel_commands()
        
        takeoff_msg = EmptyTopicMsg()
        print "Taking-Off Start"
        self.takeoff_pub.publish(takeoff_msg)
        time.sleep(seconds_taking_off)
        print "Taking-Off sequence completed"
        

    def improved_distance_reward(self, current_pose):
        current_dist = self.calculate_dist_between_two_Points(current_pose.position, self.desired_pose.position)
        rospy.loginfo("Calculated Distance = "+str(current_dist))
        
        if current_dist < self.best_dist:
            reward = 100
            self.best_dist = current_dist
        elif current_dist == self.best_dist:
            reward = 0
        else:
            reward = -100
            print "Made Distance bigger= "+str(self.best_dist)
        
        return reward
        
    def process_data(self, data_position, data_imu):

        max_altitude = 2.0
        done = False
        max_incl = 0.7

        euler = tf.transformations.euler_from_quaternion([data_imu.orientation.x,data_imu.orientation.y,data_imu.orientation.z,data_imu.orientation.w])
        roll = euler[0]
        pitch = euler[1]
        yaw = euler[2]

        pitch_bad = not(-max_incl < pitch < max_incl)
        roll_bad = not(-max_incl < roll < max_incl)
        altitude_bad = data_position.position.z > max_altitude

        if altitude_bad or pitch_bad or roll_bad:
            print ("(TOO_HIGH,TOO MUCH PITCH, TOO MUCH ROLL) >>> ("+str(altitude_bad)+","+str(pitch_bad)+","+str(roll_bad)+")")
            done = True
            reward = -200
        else:
            reward = self.improved_distance_reward(data_position)

        return reward,done

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _step(self, action):

        rospy.wait_for_service('/gazebo/unpause_physics')
        try:
            self.unpause()
        except rospy.ServiceException, e:
            print ("/gazebo/unpause_physics service call failed")

        speed_value = 1.0
        if action == 0: #FORWARD
            vel_cmd = Twist()
            vel_cmd.linear.x = speed_value
            vel_cmd.angular.z = 0.0
            self.vel_pub.publish(vel_cmd)
        elif action == 1: #LEFT
            vel_cmd = Twist()
            vel_cmd.linear.x = 0.05
            vel_cmd.angular.z = speed_value
            self.vel_pub.publish(vel_cmd)
        elif action == 2: #RIGHT
            vel_cmd = Twist()
            vel_cmd.linear.x = 0.05
            vel_cmd.angular.z = -speed_value
            self.vel_pub.publish(vel_cmd)
        elif action == 3: #Up
            vel_cmd = Twist()
            vel_cmd.linear.z = speed_value
            vel_cmd.angular.z = 0.0
            self.vel_pub.publish(vel_cmd)
        elif action == 4: #Down
            vel_cmd = Twist()
            vel_cmd.linear.z = -speed_value
            vel_cmd.angular.z = 0.0
            self.vel_pub.publish(vel_cmd)

        data_pose = None
        while data_pose is None:
            try:
                data_pose = rospy.wait_for_message('/drone/gt_pose', Pose, timeout=5)
            except:
                pass

        data_imu = None
        while data_imu is None:
            try:
                data_imu = rospy.wait_for_message('/drone/imu', Imu, timeout=5)
            except:
                pass

        
        rospy.wait_for_service('/gazebo/pause_physics')
        try:
            self.pause()
        except rospy.ServiceException, e:
            print ("/gazebo/pause_physics service call failed")


        reward,done = self.process_data(data_pose, data_imu)

        # Promote going forwards instead of turning
        if action == 0:
            reward += 100
        elif action == 1 or action == 2:
            reward -= 50
        elif action == 3:
            reward -= 150
        else:
            reward -= 50

        state = [data_pose.position.x]
        return state, reward, done, {}

    def _reset(self):

        # Resets the state of the environment and returns an initial observation.
        rospy.wait_for_service('/gazebo/reset_simulation')
        try:
            #reset_proxy.call()
            self.reset_proxy()
        except rospy.ServiceException, e:
            print ("/gazebo/reset_simulation service call failed")

        # Unpauses simulation to make observation
        rospy.wait_for_service('/gazebo/unpause_physics')
        try:
            #resp_pause = pause.call()
            self.unpause()
        except rospy.ServiceException, e:
            print ("/gazebo/unpause_physics service call failed")

        self.check_topic_publishers_connection()
        self.init_desired_pose()
        self.takeoff_sequence()

        data_pose = None
        while data_pose is None:
            try:
                data_pose = rospy.wait_for_message('/drone/gt_pose', Pose, timeout=5)
            except:
                rospy.loginfo("/drone/gt_pose topic not ready")
                pass

        
        data_imu = None
        while data_imu is None:
            try:
                data_imu = rospy.wait_for_message('/drone/imu', Imu, timeout=5)
            except:
                rospy.loginfo("/ardrone/imu topic not ready")
                pass

        rospy.wait_for_service('/gazebo/pause_physics')
        try:
            #resp_pause = pause.call()
            self.pause()
        except rospy.ServiceException, e:
            print ("/gazebo/pause_physics service call failed")

        state = [data_pose.position.x]

        return state


<p style="background:green;color:white;">**gazebo_quadcopter.py**</p>

As explained in Chapter 2, the **QuadCopterEnv (gazebo_env.GazeboEnv)** has the same main methods that the turtlebot simulation environment had. The only differences are the sensors used, the observations returned, and the way the rewards and finished criteria are handled.

## Use the Environment just created to learn

Execute the following command to see how this works

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun gym_construct simple_qlearn_quadcopter.py

<p style="background:green;color:white;">**gazebo_quadcopter.py**</p>

In [ ]:
#!/usr/bin/env python
import gym
import gym_gazebo
import time
import numpy
import random
import time
import matplotlib
import matplotlib.pyplot as plt
import qlearn
# import liveplot
from gym import wrappers
from liveplot import LivePlot 

def render():
    render_skip = 0 #Skip first X episodes.
    render_interval = 50 #Show render of each Y episode.
    render_episodes = 10 #Show Z episodes every rendering.

    if (x%render_interval == 0) and (x != 0) and (x > render_skip):
        env.render()
    elif ((x-render_episodes)%render_interval == 0) and (x != 0) and (x > render_skip) and (render_episodes < x):
        env.render(close=True)

if __name__ == '__main__':

    env = gym.make('QuadCopter-v0')
    print "Gym Makde done"
    outdir = '/tmp/gazebo_gym_experiments'
    env = wrappers.Monitor(env, outdir, force=True)          # use this to avoid warnings
    print "Monitor Wrapper started"
    last_time_steps = numpy.ndarray(0)

    qlearn = qlearn.QLearn(actions=range(env.action_space.n),
                    alpha=0.1, gamma=0.8, epsilon=0.9)

    initial_epsilon = qlearn.epsilon

    epsilon_discount = 0.999 # 1098 eps to reach 0.1

    start_time = time.time()
    total_episodes = 3
    highest_reward = 0

    for x in range(total_episodes):
        #done = False

        cumulated_reward = 0 #Should going forward give more reward then L/R ?
        print ("Episode = "+str(x))
        observation = env.reset()
        done = False
        if qlearn.epsilon > 0.05:
            qlearn.epsilon *= epsilon_discount

        env.render()
        print "START EPISODE OBS>>>>"+str(observation)+"<<<<"
        print type(observation)
        state = ''.join(map(str, observation))
        
        max_range = 1000
        for i in range(max_range):

            # Pick an action based on the current state
            action = qlearn.chooseAction(state)
            #print ("Action Chosen"+str(action))
            # Execute the action and get feedback
            observation, reward, done, info = env.step(action)
            cumulated_reward += reward
            #print ("Reward="+str(reward))
            if highest_reward < cumulated_reward:
                highest_reward = cumulated_reward

            print "Step "+str(i)+" OBS>>>>"+str(observation)+"<<<<"
            print type(observation)
            nextState = ''.join(map(str, observation))

            qlearn.learn(state, action, reward, nextState)

            #env.monitor.flush(force=True)

            if not(done):
                #print "NOT done"
                state = nextState
                if i >= max_range:
                    done = True
                    break 
            else:
                print "DONE"
                last_time_steps = numpy.append(last_time_steps, [int(i + 1)])
                break 

        m, s = divmod(int(time.time() - start_time), 60)
        h, m = divmod(m, 60)
        print ("EP: "+str(x+1)+" - [alpha: "+str(round(qlearn.alpha,2))+" - gamma: "+str(round(qlearn.gamma,2))+" - epsilon: "+str(round(qlearn.epsilon,2))+"] - Reward: "+str(cumulated_reward)+"     Time: %d:%02d:%02d" % (h, m, s))

    #Github table content
    print ("\n|"+str(total_episodes)+"|"+str(qlearn.alpha)+"|"+str(qlearn.gamma)+"|"+str(initial_epsilon)+"*"+str(epsilon_discount)+"|"+str(highest_reward)+"| PICTURE |")

    l = last_time_steps.tolist()
    l.sort()

    #print("Parameters: a="+str)
    print("Overall score: {:0.2f}".format(last_time_steps.mean()))
    print("Best 100 score: {:0.2f}".format(reduce(lambda x, y: x + y, l[-100:]) / len(l[-100:])))

    #env.monitor.close()
    #env.close()


<p style="background:green;color:white;">**gazebo_quadcopter.py**</p>

<p style="background:#EE9023;color:white;">**Exercise 4.2**</p>

* Create your own learning script that uses your own version of QuadCopter-v1. Where should you create it? In your own workspace **catkin_ws/src**, for instance.

* Base it on the prior one, if you wish. Try to use other sensors, other rewards, and other criteria to state that the learning episode has finished.


* Change the rewards and see how the emerging behaviour changes.


* Why not use the cameras? If you feel adventurous, you could try to base the learning on images.

<p style="background:#EE9023;color:white;">**END of Exercise 4.2**</p>